In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()  

In [0]:
# !pip install split-folders

In [0]:
# unzip /content/101_ObjectCategories.zip

In [0]:
# import split_folders
# split_folders.ratio('101_ObjectCategories', output="output", seed=1337, ratio=(.6, .2, .2)) 

In [0]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = 'output'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val','test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
save_file_name = 'alexnet-transfer.pt'


# New Section

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), save_file_name)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
model = torchvision.models.alexnet(pretrained=True)
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [0]:


# Freeze early layers
for param in model.parameters():
    param.requires_grad = False



In [0]:
n_inputs = model.classifier[6].in_features
# Add on classifier
model.classifier[6] = nn.Sequential(
    nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.5),
    nn.Linear(256, 102), nn.LogSoftmax(dim=1))
#print the classifier layer
model.classifier

Sequential(
  (0): Dropout(p=0.5)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace)
  (3): Dropout(p=0.5)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=256, out_features=102, bias=True)
    (4): LogSoftmax()
  )
)

In [0]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

58,078,886 total parameters.
1,075,046 training parameters.


In [0]:
model = model.to(device)

In [0]:
# model.class_to_idx = image_datasets['train'].class_to_idx
# model.idx_to_class = {
#     idx: class_
#     for class_, idx in model.class_to_idx.items()
# }

# list(model.idx_to_class.items())[:10]

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [0]:
model_conv = train_model(model, criterion, optimizer,
                         exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 3.2906 Acc: 0.2992
val Loss: 2.7707 Acc: 0.4223

Epoch 1/24
----------
train Loss: 2.9190 Acc: 0.3781
val Loss: 2.4098 Acc: 0.4966

Epoch 2/24
----------
train Loss: 2.6267 Acc: 0.4407
val Loss: 2.1109 Acc: 0.5749

Epoch 3/24
----------
train Loss: 2.3964 Acc: 0.4783
val Loss: 1.8648 Acc: 0.6096

Epoch 4/24
----------
train Loss: 2.1979 Acc: 0.5173
val Loss: 1.6687 Acc: 0.6594

Epoch 5/24
----------
train Loss: 2.0280 Acc: 0.5524
val Loss: 1.5038 Acc: 0.6829

Epoch 6/24
----------
train Loss: 1.9378 Acc: 0.5671
val Loss: 1.4893 Acc: 0.6890

Epoch 7/24
----------
train Loss: 1.9350 Acc: 0.5647
val Loss: 1.4758 Acc: 0.6941

Epoch 8/24
----------
train Loss: 1.9390 Acc: 0.5577
val Loss: 1.4629 Acc: 0.6957

Epoch 9/24
----------
train Loss: 1.8892 Acc: 0.5782
val Loss: 1.4498 Acc: 0.6985

Epoch 10/24
----------
train Loss: 1.9215 Acc: 0.5656
val Loss: 1.4373 Acc: 0.7019

Epoch 11/24
----------
train Loss: 1.8956 Acc: 0.5652
val Loss: 1.4243 Acc: 0.7030

Ep

In [0]:
def accuracy(outputs, labels):
    """Compute the topk accuracy(s)"""

    with torch.no_grad():
        batch_size = labels.size(0)

        # Find the predicted classes and transpose
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss = loss.item() * batch_size
        running_corrects = torch.sum(preds == labels.data)
        return running_corrects.double()/batch_size

In [0]:
testiter = iter(dataloaders['test'])
# Get a batch of testing images and labels
inputs, labels = next(testiter)
accuracy(model(inputs.to(device)), labels.to(device))

tensor(0.7500, device='cuda:0', dtype=torch.float64)